In [110]:
import OpenEphys

In [111]:
%timeit OpenEphys.loadContinuous('test/100_ADC2.continuous')

Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
6.36 s ± 288 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [112]:
%timeit OpenEphys.loadContinuousFast('test/100_ADC2.continuous')

Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
771 ms ± 8.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%timeit f=open('test/100_ADC2.continuous','rb'); f.read()

114 ms ± 3.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [105]:
from OpenEphys import *

def loadContinuousFast2(filepath, dtype = float):

    assert dtype in (float, np.int16), \
        'Invalid data type specified for loadContinous, valid types are float and np.int16'

    print("Loading continuous data...")

    ch = { }

    #read in the data
    f = open(filepath,'rb')

    fileLength = os.fstat(f.fileno()).st_size

    # calculate number of samples
    recordBytes = fileLength - NUM_HEADER_BYTES
    if  recordBytes % RECORD_SIZE != 0:
        raise Exception("File size is not consistent with a continuous file: may be corrupt")
    nrec = recordBytes // RECORD_SIZE
    nsamp = nrec * SAMPLES_PER_RECORD
    # pre-allocate samples
    samples = np.zeros(nsamp, dtype)
    timestamps = np.zeros(nrec)
    recordingNumbers = np.zeros(nrec)
    indices = np.arange(0, nsamp + 1, SAMPLES_PER_RECORD, np.dtype(np.int64))

    header = readHeader(f)

    buffer = f.read()
    data_tmp=np.frombuffer(buffer,np.dtype('>i2')) #read everything into a large buffer
    data_tmp = data_tmp.reshape(int(len(b3)/(RECORD_SIZE/2)),int(RECORD_SIZE/2)) #reshape it into each chunk
    
    timestamps = data_tmp[:,:4].ravel().view('<i8') #reinterpret the timestamp
    N = data_tmp[:,4].ravel().view('<u2') #reinterpret number of recording
    recordingNumbers = data_tmp[:,5].ravel().view('>u2') #reintepret the recording number
    
    if np.any(N!=SAMPLES_PER_RECORD):
        raise Exception('Found corrupted record at '+np.where(N!=SAMPLES_PER_RECORD))
        
    if dtype == float: # Convert data to float array and convert bits to voltage.
        samples=data_tmp[:,6:6+SAMPLES_PER_RECORD].ravel() * float(header['bitVolts']) # #extract the data
    else:  # Keep data in signed 16 bit integer format.
        samples=data_tmp[:,6:6+SAMPLES_PER_RECORD].ravel()    
     

    ch['header'] = header
    ch['timestamps'] = timestamps
    ch['data'] = samples  # OR use downsample(samples,1), to save space
    ch['recordingNumber'] = recordingNumbers
    f.close()
    return ch

d=loadContinuousFast('test/100_ADC2.continuous')
d2 = loadContinuous('test/100_ADC2.continuous')

print(np.all(d['data']==d2['data']))
print(np.all(d['header']==d2['header']))
print(np.all(d['recordingNumber']==d2['recordingNumber']))
print(np.all(d['timestamps']==d2['timestamps']))


Loading continuous data...
Loading continuous data...
True
True
True
True


In [109]:
d=OpenEphys.loadContinuousFast('test/100_ADC2.continuous')
d2 = OpenEphys.loadContinuous('test/100_ADC2.continuous')

print(np.all(d['data']==d2['data']))
print(np.all(d['header']==d2['header']))
print(np.all(d['recordingNumber']==d2['recordingNumber']))
print(np.all(d['timestamps']==d2['timestamps']))

Loading continuous data...
Loading continuous data...
True
True
True
True


In [32]:
f=open('test/100_ADC2.continuous','rb'); 
b=f.read()

In [36]:
NUM_HEADER_BYTES = 1024
SAMPLES_PER_RECORD = 1024
BYTES_PER_SAMPLE = 2
RECORD_SIZE = 4 + 8 + SAMPLES_PER_RECORD * BYTES_PER_SAMPLE + 10 # size of each continuous record in bytes
#8 bytes timestamp, 2 bytes number of sample, 2 bytes recording number, 

In [40]:
b2=b[1024:]
(len(b2))/(2070)

93281.0

In [41]:
b3=np.frombuffer(b2,np.dtype('>i2'))

In [76]:
b4=b3.reshape(int(len(b3)/(RECORD_SIZE/2)),int(RECORD_SIZE/2))
b5=b4[5:SAMPLES_PER_RECORD,:]

In [79]:
b4[:,4].view('<u2')

array([1024, 1024, 1024, ..., 1024, 1024, 1024], dtype=uint16)